# 学習データを作成する

## 概要
1. 分子内の結合（単結合のみ）の切断エネルギーのデータを読み込む
2. 各結合の特徴量を計算する
3. 結合の特徴量と切断エネルギーからなる学習データを作成する

## 必要なライブラリをインポート

In [4]:
import os
import glob
import time
import json
import subprocess
import concurrent.futures
import zipfile
import pandas as pd
from rdkit import Chem

## 1. 分子内の結合（単結合のみ）の切断エネルギーのデータを読み込む
下記論文にて公開されたデータ`data/rdf_data_190531.csv`を使用する。
```
St. John, P.C., Guan, Y., Kim, Y. et al. Prediction of organic homolytic bond dissociation enthalpies at near chemical accuracy with sub-second computational cost. Nat Commun 11, 2328 (2020). https://doi.org/10.1038/s41467-020-16201-z
```

In [7]:
df = pd.read_csv('data/rdf_data_190531.csv')
df

,rid,molecule,bond_index,fragment1,fragment2,bde,bond_type
0,1,NCCCC(=O)O,0,[CH2]CCC(=O)O,[NH2],87.599001,C-N
1,2,NCCCC(=O)O,1,[CH2]CC(=O)O,[CH2]N,82.344241,C-C
2,3,NCCCC(=O)O,2,[CH2]C(=O)O,[CH2]CN,86.313863,C-C
3,4,NCCCC(=O)O,3,O=[C]O,[CH2]CCN,95.898435,C-C
4,5,NCCCC(=O)O,5,NCCC[C]=O,[OH],111.408575,C-O
...,...,...,...,...,...,...,...
484902,849913,CC(N)CN,9,[H],CC([NH])CN,98.950012,H-N
484903,849914,CC(N)CN,10,[H],CC(N)[CH]N,92.019802,C-H
484904,849915,CC(N)CN,11,[H],CC(N)[CH]N,92.019802,C-H
484905,849916,CC(N)CN,12,[H],CC(N)C[NH],99.025940,H-N


## 2. 各結合の特徴量を計算する
下記論文で公開されたツール`TypePairBondDescriptor.jar`を使用して、各結合の特徴量計算を行う。
```
Qu, X., Latino, D.A. & Aires-de-Sousa, J. A big data approach to the ultra-fast prediction of DFT-calculated bond energies. J Cheminform 5, 34 (2013). https://doi.org/10.1186/1758-2946-5-34
```  
プログラムは下記で配布されている。特徴量計算を実行する場合は、`bin/TypePairBondDescriptor.jar`としてjarを配置して、実行する。  
https://sourceforge.net/projects/unlpredict/  
※特徴量化プログラムの利用については自己責任でお願いします。  

In [30]:
# すべての分子の構造を示すSMILESを取得する。
smiles_set = set(df['molecule'].unique())
smiles_list = list(smiles_set)

In [ ]:
# 各smilesの各結合の特徴量を計算（16並列で計算）
# 非常に時間がかかる

jar_path = "bin/TypePairBondDescriptor.jar"
def run(arg):
    i = arg[0]
    smiles = arg[1]
    mol = Chem.MolFromSmiles(smiles)
    sdf_file = f'data/mol{i}.sdf'
    w = Chem.SDWriter(sdf_file)
    w.write(mol)
    w.close()
    command = ["java", "-jar", jar_path, "compact", "All", sdf_file, "true"]
    result = subprocess.run(command, capture_output=True, text=True)
    return result.stdout

def process_chunk(chunk):
    args = [[i, smiles] for i, smiles in enumerate(chunk)]
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(run, args)

        with open("data/descriptor.txt", "a") as file:
            for i, result in enumerate(results):
                file.write(chunk[i] + '\n')
                file.write(result)


start = time.time() 
chunk_size = 16
for i in range(0, len(smiles_list), chunk_size):
    chunk = smiles_list[i:i+chunk_size] 
    process_chunk(chunk)
    now = time.time()
    print(f"{i + 16} smiles done: {now - start} 秒")

In [3]:
# 特徴量計算に使用した1時ファイルを削除
sdf_files = glob.glob('data/*.sdf')
for file_path in sdf_files:
    try:
        os.remove(file_path)
    except Exception as e:
        print(f"Error deleteing {file_path}: {e}")

## 3. 結合の特徴量と切断エネルギーからなる学習データを作成する
1, 2で取得したデータから、結合の特徴量と切断エネルギーからなる学習データを作成する。  
学習データは、`data/all_train_data.csv`として出力される。

In [28]:
bond_descriptor = {}
key = None
value = None
with open('data/descriptor.txt', 'r') as f:
    for line in f:
        data = line.split()
        if len(data) == 1:
            if key is not None:
                bond_descriptor[key] = value
            key = data[0]
            value = []
            continue

        value.append(data)

In [29]:
bond_rid_descriptor = [] 
for index, row in df.iterrows():
    bond_rid_descriptor.append([row['rid'], *bond_descriptor[row['molecule']][row['bond_index']]])

In [30]:
df_descriptor = pd.DataFrame(bond_rid_descriptor)

In [31]:
df_descriptor

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,1,1,2,0,0,1,0,0,0,1,...,0,6,0,4,0,0,2,0,0,0
1,2,2,3,0,0,2,0,0,0,0,...,6,6,4,3,0,2,0,0,0,0
2,3,3,4,0,0,2,0,0,0,0,...,6,9,7,0,2,0,0,0,0,0
3,4,4,5,0,1,1,0,0,0,0,...,3,6,3,4,0,2,0,0,0,0
4,5,5,7,0,1,0,0,0,0,0,...,0,3,0,3,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484902,849913,3,11,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
484903,849914,4,12,0,0,1,1,0,0,0,...,0,10,0,0,0,0,0,0,0,0
484904,849915,4,13,0,0,1,1,0,0,0,...,0,10,0,0,0,0,0,0,0,0
484905,849916,5,14,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# 意味のない列を削除
atom_ids = [1, 2]
df_descriptor = df_descriptor.drop(atom_ids, axis=1)
df_descriptor

,0,3,4,5,6,7,8,9,10,11,...,122,123,124,125,126,127,128,129,130,131
0,1,0,0,1,0,0,0,1,0,0,...,0,6,0,4,0,0,2,0,0,0
1,2,0,0,2,0,0,0,0,0,0,...,6,6,4,3,0,2,0,0,0,0
2,3,0,0,2,0,0,0,0,0,0,...,6,9,7,0,2,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,...,3,6,3,4,0,2,0,0,0,0
4,5,0,1,0,0,0,0,0,0,0,...,0,3,0,3,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484902,849913,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
484903,849914,0,0,1,1,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
484904,849915,0,0,1,1,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
484905,849916,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# すべての行で値が同じ列を削除
uniform_columns = df_descriptor.columns[df_descriptor.nunique() == 1]
uniform_columns

Index([  7,  10,  11,  13,  14,  15,  16,  21,  24,  27,  28,  29,  30,  38,
        41,  42,  43,  44,  52,  55,  56,  57,  58,  63,  67,  70,  71,  72,
        73,  78,  82,  85,  86,  87,  88,  93,  97, 100, 101, 102, 103, 110],
      dtype='int64')

In [34]:
df_descriptor.drop(columns = uniform_columns, inplace = True)
df_descriptor

,0,3,4,5,6,8,9,12,17,18,...,122,123,124,125,126,127,128,129,130,131
0,1,0,0,1,0,0,1,0,0,0,...,0,6,0,4,0,0,2,0,0,0
1,2,0,0,2,0,0,0,0,0,0,...,6,6,4,3,0,2,0,0,0,0
2,3,0,0,2,0,0,0,0,0,1,...,6,9,7,0,2,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,...,3,6,3,4,0,2,0,0,0,0
4,5,0,1,0,0,0,0,1,0,0,...,0,3,0,3,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484902,849913,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
484903,849914,0,0,1,1,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
484904,849915,0,0,1,1,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
484905,849916,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df_all = pd.merge(df, df_descriptor, left_on='rid', right_on=0, how='left')

In [36]:
df_all

,rid,molecule,bond_index,fragment1,fragment2,bde,bond_type,0,3,4,...,122,123,124,125,126,127,128,129,130,131
0,1,NCCCC(=O)O,0,[CH2]CCC(=O)O,[NH2],87.599001,C-N,1,0,0,...,0,6,0,4,0,0,2,0,0,0
1,2,NCCCC(=O)O,1,[CH2]CC(=O)O,[CH2]N,82.344241,C-C,2,0,0,...,6,6,4,3,0,2,0,0,0,0
2,3,NCCCC(=O)O,2,[CH2]C(=O)O,[CH2]CN,86.313863,C-C,3,0,0,...,6,9,7,0,2,0,0,0,0,0
3,4,NCCCC(=O)O,3,O=[C]O,[CH2]CCN,95.898435,C-C,4,0,1,...,3,6,3,4,0,2,0,0,0,0
4,5,NCCCC(=O)O,5,NCCC[C]=O,[OH],111.408575,C-O,5,0,1,...,0,3,0,3,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484902,849913,CC(N)CN,9,[H],CC([NH])CN,98.950012,H-N,849913,0,0,...,0,0,0,0,0,0,0,0,0,0
484903,849914,CC(N)CN,10,[H],CC(N)[CH]N,92.019802,C-H,849914,0,0,...,0,10,0,0,0,0,0,0,0,0
484904,849915,CC(N)CN,11,[H],CC(N)[CH]N,92.019802,C-H,849915,0,0,...,0,10,0,0,0,0,0,0,0,0
484905,849916,CC(N)CN,12,[H],CC(N)C[NH],99.025940,H-N,849916,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# 学習に不要な列を削除
df_train = df_all.drop(['rid', 'molecule', 'bond_index', 'fragment1', 'fragment2', 'bond_type', 0], axis=1)

# 目的変数を一番右に移動
target_column = df_train.pop('bde')
df_train['bde'] = target_column
df_train

,3,4,5,6,8,9,12,17,18,19,...,123,124,125,126,127,128,129,130,131,bde
0,0,0,1,0,0,1,0,0,0,1,...,6,0,4,0,0,2,0,0,0,87.599001
1,0,0,2,0,0,0,0,0,0,1,...,6,4,3,0,2,0,0,0,0,82.344241
2,0,0,2,0,0,0,0,0,1,1,...,9,7,0,2,0,0,0,0,0,86.313863
3,0,1,1,0,0,0,0,0,0,1,...,6,3,4,0,2,0,0,0,0,95.898435
4,0,1,0,0,0,0,1,0,0,1,...,3,0,3,0,0,2,0,0,0,111.408575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484902,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,98.950012
484903,0,0,1,1,0,0,0,0,0,1,...,10,0,0,0,0,0,0,0,0,92.019802
484904,0,0,1,1,0,0,0,0,0,1,...,10,0,0,0,0,0,0,0,0,92.019802
484905,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,99.025940


In [38]:
# 学習データを出力
train_data_file = 'data/all_train_data.csv'
train_data_zip ='data/all_train_data.zip'
df_train.to_csv(train_data_file, index=False)

In [39]:
# 学習データをZIP圧縮
with zipfile.ZipFile(train_data_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(train_data_file, arcname=train_data_file)